In [1]:
#!pip install solid-file
#!pip install web3

In [8]:
# imports
from solid.solid_api import SolidAPI
from solid.auth import Auth
from getpass import getpass
from web3 import Web3, HTTPProvider
from web3.middleware import geth_poa_middleware
import json
import io

# Welcome to the Papayaverse Demo Notebook

INSERT A BUNCH OF INTRODUCTORY TEXT

## Login

Please login to solidcommunity.net

In [3]:
# LOGIN MODULE
USERNAME = input('Please enter your  solid community user name:')
PASSWORD = getpass('Please enter your solid community password:')
IDP = 'https://solidcommunity.net'
auth = Auth()
solid_connection = SolidAPI(auth)
auth.login(IDP, USERNAME, PASSWORD)

Please enter your  solid community user name:rammkripa
Please enter your solid community password:········


## Wallet


In [4]:
## Setup for Web3
web3provider = 'https://goerli.infura.io/v3/85e35e212e7c431a838571e469b3c64b'
web3_connection = Web3(HTTPProvider(web3provider))
print ("Latest Ethereum block number", web3_connection.eth.blockNumber)
web3_connection.middleware_onion.inject(geth_poa_middleware, layer=0)
if web3_connection.isConnected(): # verify that the connection worked
    print('Now Connected to the Goerli Network!')
else :
    print('An Error Occurred ;-;')

Latest Ethereum block number 8411477
Now Connected to the Goerli Network!


In [5]:
## Find Existing Wallet
wallet_folder = 'https://' + USERNAME + '.solidcommunity.net/private/wallet/'
required_files = ['provider.md', 'wallet_address.md', 'private_key.md']
criterion = all([solid_connection.item_exists(wallet_folder + i) for i in required_files])
if criterion: # Does the user already have a wallet?
    wallet_address_file_name = wallet_folder + 'wallet_address.md'
    wallet_address_resp = solid_connection.get(wallet_address_file_name)
    wallet_address = resp.text
    print('Found Existing Wallet ' + wallet_address)
## Or Create One
else :
    my_account = web3_connection.eth.account.create(USERNAME)
    wallet_address = my_account._address
    print('Created New Wallet for You ' + wallet_address)
## And display the balance
balance = web3_connection.eth.get_balance(account = wallet_address) # gets balance in wei
print('Balance is ' + str(Web3.fromWei(balance, 'ether')) + ' ether')

Created New Wallet for You 0x4Ab16d8D3Ec90DdA9Fe105F241FcaaeA01D08961
Balance is 0 ether


In [9]:
## Save wallet information if new wallet
def save_wallet_info(file_string, file_nm):
    f = io.BytesIO(file_string.encode('UTF-8'))
    file_name = wallet_folder + file_nm
    solid_connection.put_file(file_name, f, 'text/markdown')
# Save provider
save_wallet_info(web3provider, 'provider.md')
# Save wallet address
save_wallet_info(wallet_address, 'wallet_address.md')
# Save private key
private_key = my_account._private_key
save_wallet_info(private_key, 'private_key.md')

AttributeError: 'HexBytes' object has no attribute 'encode'